In [1]:
import trimesh
import numpy as np
from io_utils import stdout_redirected
import time
from dvidutils import encode_faces_to_custom_drc_bytes
import dask
import utils

from collections import namedtuple 
import openmesh as om
import sys
import os

Fragment = namedtuple('Fragment', ['draco_bytes', 'position','offset'])

def get_face_indices_in_range(mesh, face_mins, stop):
    max_edge_length = np.max(mesh.edges_unique_length)
    rows = np.where(face_mins[:,0]<stop+max_edge_length)
    return rows[0]

def renumber_vertex_indices(faces, vertex_indices_in_range):
    def renumber_indicies(a, val_old, val_new):
        arr = np.empty(a.max()+1, dtype=val_new.dtype)
        arr[val_old] = val_new
        return arr[a]
    
    faces = np.reshape(faces,-1)
    faces=renumber_indicies(faces,vertex_indices_in_range,np.arange(len(vertex_indices_in_range)))

    return np.reshape(faces,(-1,3))

def my_slice_faces_plane(v,f,plane_normal, plane_origin):
    # Wrapper for trimesh slice_faces_plane, checks that there are vertices and faces and catches an error that happens if the whole mesh is to one side

    if len(v)>0 and len(f)>0:
        try:
            v, f = trimesh.intersections.slice_faces_plane(
                    v, f, plane_normal=plane_normal, plane_origin=plane_origin)
        except ValueError as e:
            if str(e) != "input must be 1D integers!":
                raise
            else:
                pass

    return v,f


@dask.delayed
def generate_mesh_decomposition(v,f,box_size, start_fragment, end_fragment, x_start, x_end):
    # Scale our coordinates.

    nyz, nxz, nxy = np.eye(3)

    fragments = []
    for x in range(x_start, x_end):
        vx, fx = my_slice_faces_plane(
            v, f, plane_normal=-nyz, plane_origin=nyz*(x+1)*box_size)

        for y in range(start_fragment[1], end_fragment[1]):
            vy, fy = my_slice_faces_plane(vx, fx, plane_normal=-nxz, plane_origin=nxz*(y+1)*box_size)

            for z in range(start_fragment[2], end_fragment[2]):
                vz, fz = my_slice_faces_plane(
                    vy, fy, plane_normal=-nxy, plane_origin=nxy*(z+1)*box_size)

                if len(vz) > 0:
                    draco_bytes = encode_faces_to_custom_drc_bytes(
                        vz, np.zeros(np.shape(vz)), fz, np.asarray(3*[box_size]), np.asarray([x,y,z])*box_size, position_quantization_bits = 10)
                    
                    if len(draco_bytes)>12:
                        fragment = Fragment(draco_bytes, [x, y, z], len(draco_bytes))   
                        fragments.append(fragment)

                vy, fy = my_slice_faces_plane(
                    vy, fy, plane_normal=nxy, plane_origin=nxy*(z+1)*box_size)

            vx, fx = my_slice_faces_plane(
                vx, fx, plane_normal=nxz, plane_origin=nxz*(y+1)*box_size)
        
        v, f = my_slice_faces_plane(
            v, f, plane_normal=nyz, plane_origin=nyz*(x+1)*box_size)
    return fragments

def decimate(v, f, fraction):
    target = max(4, int(fraction * len(v)))

    try:
        sys.stderr.fileno()
    except:
        # Can't redirect stderr if it has no file descriptor.
        # Just let the output spill to wherever it's going.
        m = om.TriMesh(v, f)
    else:
        # Hide stderr, since OpenMesh construction is super noisy.
        with stdout_redirected(stdout=sys.stderr):
            m = om.TriMesh(v, f)
        
        h = om.TriMeshModQuadricHandle()
        d = om.TriMeshDecimater(m)
        d.add(h)
        d.module(h).unset_max_err()
        d.initialize()

        print(
            f"Attempting to decimate to {target} (Reduce by {len(mesh.vertices) - target})")
        eliminated_count = d.decimate_to(target)
        print(f"Reduced by {eliminated_count}")
        m.garbage_collection()

    v = m.points().astype(np.float32)
    f = m.face_vertex_indices().astype(np.uint32)

    return v,f

In [2]:
mesh = trimesh.load(
        "/groups/cosem/cosem/ackermand/meshesForWebsite/res1decimation0p1/jrc_hela-1/er_seg/1.obj")
print("read")
print(np.min(mesh.vertices, axis=0))
print(np.max(mesh.vertices, axis=0))

v,f = trimesh.intersections.slice_faces_plane(mesh.vertices, mesh.faces, [0,0,1],[-1000000,-1000000,-1000000])

read
[39830.8   3303.42  5457.47]
[45119.7   4031.32  9730.05]


In [3]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4,
                n_workers=16)


distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-9jh0jsva', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-6biczx_i', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-r4xmzbdw', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-03v6vmp7', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-r342prr7', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ac

In [4]:
import dask

nyz, nxz, nxy = np.eye(3)
num_workers = 16
num_lods = 1

lods = list(range(num_lods+1))


v_whole = mesh.vertices.astype(np.float32)
f_whole = mesh.faces.astype(np.uint32)

lod_0_box_size = 64*4

for current_lod in lods:
    results = []

    v = v_whole
    f = f_whole

    if len(lods)>1: #decimate here so don't have to keep original v_whole, f_whole around
        v_whole,f_whole = decimate(v_whole, f_whole, 0.5)

    box_size = lod_0_box_size*2**current_lod

    start_fragment = np.min(v, axis=0).astype(int) // box_size - 1
    end_fragment =  np.max(v, axis=0).astype(int) // box_size + 1
    x_stride = int(np.ceil(1.0*(end_fragment[0]-start_fragment[0])/num_workers))

    for x in range(start_fragment[0], end_fragment[0]+1,x_stride):
        vx, fx = my_slice_faces_plane(v, f, plane_normal=-nyz, plane_origin=nyz*(x+x_stride)*box_size)

        results.append( generate_mesh_decomposition(vx, fx, box_size, start_fragment, end_fragment, x, x+x_stride) )

        v, f = my_slice_faces_plane(v, f, plane_normal=nyz, plane_origin=nyz*(x+x_stride)*box_size)

    dask_results = dask.compute(* results)
    fragments = [fragment for fragments in dask_results for fragment in fragments]
    utils.write_files("test/simpler_multires", "1", fragments, current_lod, lods, np.asarray([box_size,box_size,box_size]))


PolyMeshT::add_face: patch re-linking failed
PolyMeshT::add_face: patch re-linking failed
PolyMeshT::add_face: complex edge
PolyMeshT::add_face: complex edge
PolyMeshT::add_face: patch re-linking failed
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex edge
PolyMeshT::add_face: complex edge
PolyMeshT

Attempting to decimate to 4499 (Reduce by 4500)
Reduced by 4500
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearray'>
<class 'bytearra